# Student Loan Risk with Deep Learning

In [34]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path
from tensorflow.keras.models import load_model
import numpy as np

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [5]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [6]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop('credit_ranking', axis=1)

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [7]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [8]:
# Create a StandardScaler instance
scaler = StandardScaler().fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = scaler.transform(X_train)

# Fit the scaler to the features training dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [9]:
# Define the the number of inputs (features) to the model
input_nodes = len(X.columns)

# Review the number of features
input_nodes

11

In [13]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_nodes = 1

In [14]:
# Create the Sequential model instance
nn_model = tf.keras.models.Sequential()

# Add the first hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer1, activation="relu", input_dim=input_nodes))

# Add the second hidden layer
nn_model.add(tf.keras.layers.Dense(hidden_nodes_layer2, activation="relu"))

# Add the output layer to the model specifying the number of output neurons and activation function
nn_model.add(tf.keras.layers.Dense(output_nodes, activation="sigmoid"))

In [15]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 6)                 72        
                                                                 
 dense_4 (Dense)             (None, 3)                 21        
                                                                 
 dense_5 (Dense)             (None, 1)                 4         
                                                                 
Total params: 97 (388.00 Byte)
Trainable params: 97 (388.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [16]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [17]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 2ms/step - loss: 1.0030 - accuracy: 0.4996
Epoch 2/50
38/38 [==============================] - 0s 2ms/step - loss: 0.8231 - accuracy: 0.5421
Epoch 3/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7726 - accuracy: 0.5621
Epoch 4/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7394 - accuracy: 0.5863
Epoch 5/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7231 - accuracy: 0.5872
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.7081 - accuracy: 0.5947
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6931 - accuracy: 0.5922
Epoch 8/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6842 - accuracy: 0.5972
Epoch 9/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6773 - accuracy: 0.5963
Epoch 10/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6696 - accuracy: 0.6013
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [18]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5849 - accuracy: 0.6750 - 250ms/epoch - 19ms/step
Loss: 0.5849040746688843, Accuracy: 0.675000011920929


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [26]:
# Set the model's file path
file_path = Path(r"C:\Users\MiChaElLa\Desktop\Columbia - BOOTCAMP\18 Module\neural-network-challenge-1\student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

In [27]:
# Set the model's file path
file_path = Path(r"C:\Users\MiChaElLa\Desktop\Columbia - BOOTCAMP\18 Module\neural-network-challenge-1\student_loans.keras")

# Load the model to a new object
loaded_model = load_model(file_path)


### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [30]:
# Make predictions with the test data
predictions = nn_model.predict(X_test,verbose=2)


# Display a sample of the predictions
print(predictions[:10])

13/13 - 0s - 47ms/epoch - 4ms/step
[[0.5936483 ]
 [0.36099994]
 [0.6960592 ]
 [0.6104646 ]
 [0.9013651 ]
 [0.90928835]
 [0.8587961 ]
 [0.1767498 ]
 [0.4994777 ]
 [0.49460396]]


In [36]:
# Round the predictions to binary results (0 or 1)
binary_predictions = np.round(predictions).astype(int)
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(binary_predictions, columns=['Predictions'])

# Display a sample of the predictions DataFrame
predictions_df.head(10)

,Predictions
0,1
1,0
2,1
3,1
4,1
5,1
6,1
7,0
8,0
9,0


### Step 4: Display a classification report with the y test data and predictions

In [38]:
# Print the classification report with the y test data and predictions
# Convert the binary predictions to 1D array for classification report
binary_predictions_1d = np.squeeze(binary_predictions)

print(classification_report(y_test, binary_predictions_1d))

              precision    recall  f1-score   support

           0       0.67      0.61      0.64       188
           1       0.68      0.74      0.71       212

    accuracy                           0.68       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.67      0.68      0.67       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

To build a recommendation system for student loan options, relevant data might include:

*   Student Profile Data: Information about the student's demographics (age, gender, location), educational background (school, GPA), financial status (income, assets), and career aspirations.
*   Loan Options Data: Details about available loan products, including interest rates, terms, repayment options, and eligibility criteria.
*   Historical Loan Performance Data: Data on past borrowers, including repayment behavior, defaults, and loan outcomes.
*   Preferences and Feedback Data: Feedback from students about their loan experiences, preferences, and satisfaction levels.

This data would be relevant and appropriate because it provides insights into students' financial needs, preferences, and behaviors. By analyzing this data, the recommendation system can tailor loan options to match individual student profiles, increasing the likelihood of loan acceptance and successful repayment.

**Choice of Filtering Method:**
Based on the selected data, the recommendation system might use a combination of collaborative filtering and content-based filtering. Collaborative filtering can leverage historical loan performance data to identify similar borrowers and recommend loan options based on their past behaviors. Content-based filtering can utilize student profile data and loan characteristics to suggest options that align with each student's preferences and financial situation.

**Real-world Challenges:**
> Privacy and Data Security: Collecting sensitive student data raises concerns about privacy and data security. Students may be hesitant to share personal information, especially financial details, due to privacy risks. Ensuring robust data protection measures and obtaining explicit consent for data usage are crucial to address these concerns.
> Bias and Fairness: Recommendation systems may inadvertently perpetuate biases present in the data, leading to unequal treatment or discrimination against certain groups of students. For example, if historical loan data reflects biased lending practices, the recommendation system may reinforce these biases by favoring certain demographics or penalizing others. Mitigating bias requires careful data preprocessing, algorithmic transparency, and regular monitoring for fairness.
Addressing these challenges is essential to build a trustworthy and effective recommendation system that helps students make informed decisions about their loan options while safeguarding their privacy and promoting fairness.